In [66]:
!pip install geopandas cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
import geopandas as gpd
from geopy.geocoders import Nominatim
import cartopy.crs as ccrs
import plotly.express as px
import pandas as pd

In [68]:
# Read in the flights data
flights_data = pd.read_csv("/content/flight_data.csv")

In [69]:
flights_data = flights_data[flights_data["Routing"]=="Non Stop"]
#flights_data = flights_data.reset_index()
#flights_data.head()

In [70]:
flight_origins = set(flights_data["Origin"].unique())
flight_destinations = set(flights_data["Destination"].unique())
# Get all flight locations
all_flight_locations = list(set(list(flight_origins) + list(flight_destinations)))

In [90]:
all_flight_locations_coordinates = {}

geolocator = Nominatim(user_agent="geoapiExercises")

for location in all_flight_locations:
    all_flight_locations_coordinates[location] = geolocator.geocode(location)

GeocoderInsufficientPrivileges: ignored

In [76]:
all_flight_locations_coordinates_df = pd.DataFrame(columns=['City', 'Latitude', 'Longitude'])

i=0
for key in all_flight_locations_coordinates.keys():
    all_flight_locations_coordinates_df.loc[i] = {'City': key, 'Latitude': all_flight_locations_coordinates[key].latitude, 'Longitude': all_flight_locations_coordinates[key].longitude}
    i += 1

all_flight_locations_coordinates_df.head()

,City,Latitude,Longitude


In [77]:
path = gpd.datasets.get_path('naturalearth_lowres')
map_borders = gpd.read_file(path)

In [83]:
# Create a list of dictionaries representing the flight paths
flight_paths = []
color_idx = 0
for _, row in all_flight_traces.iterrows():
    flight_paths.append(dict(
        type='scattermapbox',
        lat=[row['Origin_Latitude'], row['Destination_Latitude']],
        lon=[row['Origin_Longitude'], row['Destination_Longitude']],
        mode='lines',
        hovertext=row['Destination'],
        line=dict(width=1, color=px.colors.qualitative.Safe[color_idx]),
        showlegend=True
    ))
    color_idx += 1
    color_idx %= len(px.colors.qualitative.Safe)

# Create the Plotly figure
fig = go.Figure()

# Add the markers to the figure
fig.add_trace(
    go.Scattermapbox(
        lat=all_flight_locations_coordinates_df['Latitude'],
        lon=all_flight_locations_coordinates_df['Longitude'],
        mode='markers',
        marker=dict(size=10, opacity=0.8, color=px.colors.qualitative.Safe),
        text=all_flight_locations_coordinates_df['City'],
        showlegend=False
    )
)

# Add the flight paths to the figure
for path in flight_paths:
    fig.add_trace(path)

# Update the layout
fig.update_layout(
    hovermode='closest',
    title={
        'text': 'Flight Paths and Locations',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24, 'color': 'black'}
    },
    margin=dict(l=20, r=20, t=50, b=20),
    mapbox=dict(
        style="carto-positron",
        center={"lat": 20, "lon": 78},
        zoom=3.5
    )
)

# Update the legend
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        font=dict(size=16, color='black')
    )
)

# Update the marker size and opacity
fig.update_traces(marker=dict(size=8, opacity=0.8))

# Show the plot
fig.show()
